In [1]:
import pandas as pd
from utils import bootstrap, youden

/opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [ ]:
regression = pd.read_csv(r'EF_predictions_regression.csv')
bootstrap_reg_new()



In [3]:
ef_35_binary = pd.read_csv(
    '/workspace/Amey/wandb_runs_and_manifests/wandb_runs/lvef_ekg_wandb/EF_binary_35/data/test_predictions.csv')

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3397: DtypeWarning: Columns (119,120,332,477,479,483,487,489,490,491,496,497,504,507,508,509,510) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
ef_35_binary = ef_35_binary[0:1000]

In [4]:
bootstrap(ef_35_binary.low_EF_binary, ef_35_binary.preds, 0.08) #0.08 is the Youden index of the EF < 35 classification model


100%|██████████| 10000/10000 [01:07<00:00, 148.60it/s]

auc: 0.878 [0.871, 0.884]
sensitivity: 0.437 [0.419, 0.454]
specificity: 0.959 [0.956, 0.961]
ppv: 0.628 [0.607, 0.648]
npv: 0.914 [0.91, 0.918]


([0.871, 0.884], [0.607, 0.648])

In [ ]:
## Regression and Binary Models - 6 bins for Classification and 6 equal Buckets for Regression

predictions = pd.read_csv('EF_predictions_regression')
labels_6 = ['0-1/6','1/6-1/3','1/3-1/2','1/2-2/3','2/3-5/6','5/6-1']

predictions['EF_preds_bins'] = pd.cut(predictions.EF_preds, np.linspace(10,70,7), True, labels_6)
data_reg = pd.DataFrame()

for i in range(0,len(labels_6)):
    if i == 0:
        data_reg = pd.DataFrame(list(predictions[predictions.EF_preds_bins.isin([labels_6[i]])].EF_2D),
                                  columns = ['True_EF'])
        data_reg['quantile'] = labels_6[i]
        
    else:
        temp = pd.DataFrame(data = list(predictions[predictions.EF_preds_bins.isin([labels_6[i]])].EF_2D),
                            columns = ['True_EF'])
        temp['quantile'] = labels_6[i]
        data_reg = pd.concat([data_reg, temp], axis = 0)
        
data_reg['Model'] = 'Regression'

ef_35_binary = pd.read_csv('EF_predictions_classification.csv')
ef_35_binary.preds = 1- ef_35_binary.preds.apply(sigmoid)
ef_35_binary['EF_preds_bins'] = pd.cut(ef_35_binary.preds,
                                       np.quantile(ef_35_binary.preds,np.linspace(
                                           0,1,7)), True, labels_6)
data_binary = pd.DataFrame()

quantile_order = labels_6

for i in range(0,len(labels_6)):
    if i == 0:
        data_binary = pd.DataFrame(list(ef_35_binary[ef_35_binary.EF_preds_bins.isin([labels_6[i]])].EF_2D),
                                  columns = ['True_EF'])
        data_binary['quantile'] = labels_6[i]
        
    else:
        temp = pd.DataFrame(data = list(ef_35_binary[ef_35_binary.EF_preds_bins.isin([labels_6[i]])].EF_2D),
                            columns = ['True_EF'])
        temp['quantile'] = labels_6[i]
        data_binary = pd.concat([data_binary, temp], axis = 0)

data_binary['Model'] = 'Classification'

data = pd.concat([data_reg, data_binary])

fig = plt.figure(figsize=(50,50))
ax = plt.axes()
ax.spines['top'].set_visible(False)
ax.spines['top'].set_color('black')
ax.spines['right'].set_visible(False)
ax.spines['right'].set_color('black')
# ax.spines['left'].set_visible(False)
# ax.spines['bottom'].set_visible(False)

plt.tick_params(axis='y', which='major', pad=30)  # Adjust the 'pad' value as needed
plt.rcParams['axes.linewidth'] = 10



sns.boxplot(data=data, x = 'quantile', y = 'True_EF',
            hue = 'Model', order = quantile_order, width=0.8, showfliers=False, linewidth=10)

ax.yaxis.set_label_coords(-.35, 0.5)

# sns.set(font_scale = 11)

# plt.plot([0,1],[0,100])

plt.title('External Validation', size = 150, fontweight = 'bold')
plt.xlabel('\nPredicted EF Intervals /\nProbability Quantiles', size = 130, fontweight = 'bold')
plt.ylabel('True Left Ventricular \nEjection Fraction', size = 130, rotation = 0, fontweight = 'bold')

plt.legend(loc = 'lower right', fontsize = 110, frameon=False)

plt.xticks([],fontsize=100)
plt.yticks(np.linspace(20,80,4),fontsize=100, rotation=0)
